# Feature Management


In [10]:
#corrigir... ver as distribuições de cada feature e corrigir

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# data partition
from sklearn.model_selection import train_test_split

#scaling methods
# minmax
# standard
# robust
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

#filter methods
# spearman 
# chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

# embedded methods
from sklearn.linear_model import LassoCV

# model
from sklearn.tree import DecisionTreeClassifier

# metric
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [21]:
learn_clean = pd.read_csv('datasetlearn_cleaned.csv', sep = ',', index_col = 0)
display(learn_clean.shape)
learn_clean.head()

(5196, 15)

,ambient_humidity,baking_duration,cooling_period,cream_fat_content,egg_temperature,egg_yolk_count,final_temperature,lemon_zest_ph,origin,oven_temperature,preheating_time,salt_ratio,sugar_content,vanilla_extract,quality_class_binary
id,,,,,,,,,,,,,,,
1,54.0,24.0,26.0,100.4,52.0,11.0,309.0,3.2,Lisboa,227.0,207.0,42.74,22.8,5.7,0
2,66.0,37.0,34.0,98.0,46.0,10.0,317.0,3.3,Lisboa,306.0,245.0,41.73,11.6,4.0,0
3,41.0,30.0,19.0,99.3,53.0,10.0,130.0,3.4,Porto,121.0,186.0,75.10,20.3,7.5,1
4,62.0,24.0,48.0,98.0,115.0,9.0,354.0,3.3,Lisboa,357.0,186.0,46.41,73.3,4.2,1
5,55.0,21.0,34.0,100.1,48.0,9.0,211.0,3.0,Lisboa,202.0,218.0,56.52,80.1,6.0,0


In [22]:
learn_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5196 entries, 1 to 5200
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ambient_humidity      5196 non-null   float64
 1   baking_duration       5196 non-null   float64
 2   cooling_period        5196 non-null   float64
 3   cream_fat_content     5196 non-null   float64
 4   egg_temperature       5196 non-null   float64
 5   egg_yolk_count        5196 non-null   float64
 6   final_temperature     5196 non-null   float64
 7   lemon_zest_ph         5196 non-null   float64
 8   origin                5196 non-null   object 
 9   oven_temperature      5196 non-null   float64
 10  preheating_time       5196 non-null   float64
 11  salt_ratio            5196 non-null   float64
 12  sugar_content         5196 non-null   float64
 13  vanilla_extract       5196 non-null   float64
 14  quality_class_binary  5196 non-null   int64  
dtypes: float64(13), int64(1), 

In [23]:
X = learn_clean.drop('quality_class_binary',axis = 1)
y = learn_clean['quality_class_binary']

In [24]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X,y, test_size=0.2, random_state=42,stratify=y  )

In [25]:

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size= 0.25, random_state= 15, shuffle= True, stratify= y_train_val)

In [26]:
l_numerical = X_train.select_dtypes(include=['int64', 'float64']).columns
X_train_n = X_train[l_numerical]
X_val_n = X_val[l_numerical]

### **fazer o one hot encoding à coluna da origin**